# A comparison of CAO points between 2019, 2020, and 2021.
***

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests as rq
import re
import datetime as dt
import urllib.request as urlrq

- How to load CAO points information from the CAO website.
- A detailed comparison of CAO points in 2019, 2020, and 2021 using pandas

<br>

## Obtaining the 2019 data from a html webpage..

In [2]:
# Pulling raw data from CAO - level 8 (2021) [1]
CAO_2021 = rq.get("http://www2.cao.ie/points/l8.php")


In [3]:
# Identifying if they pull correctly (Response [200] means its good.)
CAO_2021

<Response [200]>

In [4]:
# Compiling the regular expression code to find lines of courses. [2] There should be 949 of them. 
# Third group is ran using the 'OR' function i.e. #123* OR #123 OR # OR 123* etc.. [3]
# RE is almost there however not capturing courses with no points assigned. Needs amendment.

group1 = "([A-Z]{2}[0-9]{3})  "  # 2 uppercase letters and 3 numbers
group2 = "(.*[^[0-9]{3}|[A-Z]{3}|[#]|[\*]])  " # everything except the following items.
group3 = "*([#]|[#][0-9]{3}[\*]|[#][0-9]{3}|[0-9]{3}|[0-9]{3}[\*]|[\*]|[ *])  " # 3rd pattern powered by OR
group4 = "(.*)"

re_course = re.compile(group1 + group2 + group3 + group4)  # compling the REGEX. 

In [5]:
# Getting the date and time as items in an array.
now = dt.datetime.now()

# Taking the string items and putting them in a string.
strnow = now.strftime('%Y_%m_%d_%H%M')

In [6]:
# Creating a path to backup original html data with time/date.
original_data = "data/cao2021_raw_"+strnow+".html"

# Defining a path to output the 2021 data with time/date. 
path = "data/cao2021_"+strnow+".csv"

In [7]:
# Backing up the original html file.

# Need to change the encoder to cp1252. 
CAO_2021.encoding = "cp1252" 

# Writing raw data to the path defined above - "original-data".
with open (original_data, "w") as f:
    f.write(CAO_2021.text)

In [8]:
counter = 0  

# Writing lines from RE to a CSV file. 
with open (path, "w") as f:  

# Running a loop to find lines in the above RE. 
    for line in CAO_2021.iter_lines(): 
        decoded = line.decode('cp1252') # error in the standard used. ISO standard didn't reckonise character '\x96' on CM002.
        if re_course.fullmatch(decoded):
            CSV_2021 = re_course.sub(r"\1,\2,\3,\4", decoded)  #seperating the groups by a comma. 
            f.write(CSV_2021 + "\n") # writing each line to the path. 
            counter +=1
            
print ("\nNumber of level 8 courses in 2021: {}\n".format(counter))


Number of level 8 courses in 2021: 949



In [9]:
# Defining column names for 2021 data.
header_list = ["Course Code", "Course Name", "Round 1", "Round 2"] # ref: [12]

# Reading in data to dataframe. 
df21 = pd.read_csv(path, names=header_list)

In [10]:
df21

,Course Code,Course Name,Round 1,Round 2
0,AL801,Software Design for Virtual Reality and Gaming...,300,
1,AL802,Software Design in Artificial Intelligence for...,313,
2,AL803,Software Design for Mobile Apps and Connected ...,350,
3,AL805,Computer Engineering for Network Infrastructur...,321,
4,AL810,Quantity Surveying ...,328,
...,...,...,...,...
944,WD211,Creative Computing ...,270,
945,WD212,Recreation and Sport Management ...,262,
946,WD230,Mechanical and Manufacturing Engineering ...,230,230
947,WD231,Early Childhood Care and Education ...,266,


<br>

## Obtaining the 2020 data from an Excel file.

The 2020 CAO data is on an excel spreadsheet on the CAO website.  After backing the original data up, I will read it in using Pandas.

In [11]:
# Creating a path from original data backup.
original_data = "data/cao2020_raw_"+strnow+".xlsx"

# Using urllib function pull the original excel file and back up to the path above.
urlrq.urlretrieve("http://www2.cao.ie/points/CAOPointsCharts2020.xlsx", original_data)

('data/cao2020_raw_2021_11_15_2213.xlsx',
 <http.client.HTTPMessage at 0x24ab1dee400>)

In [12]:
# Reading in the excel file to a pandas dataframe, skipping first 10 rows.
df = pd.read_excel("http://www2.cao.ie/points/CAOPointsCharts2020.xlsx", skiprows=10)

# Creating new dataframe with only the relevant columns.
CAO_2020 = df[["COURSE CODE2","COURSE TITLE", "R1 POINTS", "R2 POINTS"]] # Ref: [10]

In [13]:
# printing the dataframe and checking the number of rows.  Excel file has 1464. 
CAO_2020

,COURSE CODE2,COURSE TITLE,R1 POINTS,R2 POINTS
0,AC120,International Business,209,NaN
1,AC137,Liberal Arts,252,NaN
2,AD101,"First Year Art & Design (Common Entry,portfolio)",#+matric,NaN
3,AD102,Graphic Design and Moving Image Design (portfo...,#+matric,NaN
4,AD103,Textile & Surface Design and Jewellery & Objec...,#+matric,NaN
...,...,...,...,...
1459,WD208,Manufacturing Engineering,188,NaN
1460,WD210,Software Systems Development,279,NaN
1461,WD211,Creative Computing,271,NaN
1462,WD212,Recreation and Sport Management,270,NaN


In [14]:
# Spot checking the dataframe
CAO_2020.iloc[[123, 1460]]

# Item at row 123 on df should be row 135 on original excel file - (CK401). 
# Item at row 1460 on df should be row 1472 on orignal excel file - (WD210).

# Minus 10 from skipped rows.
# Minus 1 as we don't include column names.
# Minus 1 as df starts at row 0, excel starts at row 1. 

,COURSE CODE2,COURSE TITLE,R1 POINTS,R2 POINTS
123,CK401,Computer Science,468,NaN
1460,WD210,Software Systems Development,279,NaN


In [15]:
# Saving the relevant CAO 2020 data to a CSV file.
path_2019 = "data/cao2020_" + strnow + ".csv"
CAO_2020.to_csv(path_2019)

Theres a lot more courses in 2020.  Need to consider whether to include addional courses with no points assigned. 
or consider only analysing the courses with points assigned across 3 years. 

<br>

## Obtaining the 2019 data from a PDF file.

### Steps to prepare the CAO 2019 data.

1. Save the PDF file to the local data folder.
2. Open in Microsoft Word and save original version of the word doc.
3. Save a 'workings' copy.
4. Delete header, footer and preamble.
5. Copy all data and paste into Notepad ++
6. Remove HEI name headings, and blank lines.
7. Find "\t\t" and replace with "\t".
8. Replace backtick "`" with single quote "'"
9. Read in to Jupyter Notebook with sep='t' and only first 4 columns.
10. Fix any errors on the CSV file. 


In [18]:
# Reading in 2019 Data. 
df2019 = pd.read_csv(r"data\cao2019_dataset.csv", sep='\t', usecols =[0,1,2,3]) # ref: [11]

In [19]:
df2019

,Course Code,INSTITUTION and COURSE,EOS,Mid
0,AL801,Software Design with Virtual Reality and Gaming,304,328.0
1,AL802,Software Design with Cloud Computing,301,306.0
2,AL803,Software Design with Mobile Apps and Connected...,309,337.0
3,AL805,Network Management and Cloud Infrastructure,329,442.0
4,AL810,Quantity Surveying,307,349.0
...,...,...,...,...
925,WD200,Arts (options),221,296.0
926,WD210,Software Systems Development,271,329.0
927,WD211,Creative Computing,275,322.0
928,WD212,Recreation and Sport Management,274,311.0


# References
- [1] http://www.cao.ie/index.php?page=points&p=2021
- [2] https://www.w3schools.com/python/python_regex.asp
- [2] https://realpython.com/regex-python/
- [3] https://www.ocpsoft.org/tutorials/regular-expressions/or-in-regex/
- https://www.w3schools.com/python/python_functions.asp
- https://stackoverflow.com/questions/54496411/python-errortypeerror-findall-missing-1-required-positional-argument-stri
- https://www.w3schools.com/python/python_regex.asp
- https://realpython.com/regex-python/
- https://stackoverflow.com/questions/2013124/regex-matching-up-to-the-first-occurrence-of-a-character
- https://developers.google.com/edu/python/regular-expressions
- https://www.ocpsoft.org/tutorials/regular-expressions/or-in-regex/
- [10] https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html
- [11] https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html
- [12] https://www.kite.com/python/answers/how-to-set-column-names-when-importing-a-csv-into-a-pandas-dataframe-in-python

***
# End